In [1]:
from mbientlab.metawear import MetaWear, libmetawear, parse_value
import mbientlab.metawear.cbindings as cbindings
from ctypes import Structure, c_void_p, cast, POINTER, c_int, c_float
from threading import Event
from time import sleep
import pandas as pd

In [2]:
# MAC address of the MetaWear device
device_address = 'D4:06:BD:85:8F:23'


# Create and connect the device
device = MetaWear(device_address)
device.connect()

WarbleException: Failed to discover device (FromBluetoothAddressAsync returned nullptr)

In [3]:
e = Event()
d = device
# Get the accelerometer data signal
signal = libmetawear.mbl_mw_acc_get_acceleration_data_signal(d.board)
# Create a logger
logger = create_voidp(lambda fn: libmetawear.mbl_mw_datasignal_log(signal, None, fn), resource = "acc_logger")

# Start logger
libmetawear.mbl_mw_logging_start(d.board, 0)

# Turn on the accelerometer
libmetawear.mbl_mw_acc_enable_acceleration_sampling(d.board)
libmetawear.mbl_mw_acc_start(d.board)

print("Logging data for 5s")
sleep(5.0)

# Turn off the accelerometer
libmetawear.mbl_mw_acc_stop(d.board)
libmetawear.mbl_mw_acc_disable_acceleration_sampling(d.board)
# Stop logging
libmetawear.mbl_mw_logging_stop(d.board)

print("Downloading data")
# Callback function to handle logger entries as we download them
def progress_update_handler(context, entries_left, total_entries):
    if (entries_left == 0):
        e.set()

# Function pointer and handlers for the logger download (LogDownloadHandler -> in the cbindings)
fn_wrapper = FnVoid_VoidP_UInt_UInt(progress_update_handler)
download_handler = LogDownloadHandler(context = None, received_progress_update = fn_wrapper, received_unknown_entry = cast(None, FnVoid_VoidP_UByte_Long_UByteP_UByte), received_unhandled_entry = cast(None, FnVoid_VoidP_DataP))
callback = FnVoid_VoidP_DataP(lambda ctx, p: print("{epoch: %d, value: %s}" % (p.contents.epoch, parse_value(p))))

# Stop logger
libmetawear.mbl_mw_logger_subscribe(logger, None, callback)

# Download logger contents
libmetawear.mbl_mw_logging_download(d.board, 0, byref(download_handler))
e.wait()

NameError: name 'create_voidp' is not defined

In [9]:
device.on_disconnect = lambda status: print ("we are disconnected!")
device.disconnect()

In [1]:
from __future__ import print_function
from mbientlab.metawear import MetaWear, libmetawear, parse_value
from mbientlab.metawear.cbindings import *
from time import sleep
from threading import Event

import platform
import sys

if sys.version_info[0] == 2:
    range = xrange

class State:
    # init state
    def __init__(self, device):
        self.device = device
        self.samples = 0
        self.accCallback = FnVoid_VoidP_DataP(self.acc_data_handler)
        self.gyroCallback = FnVoid_VoidP_DataP(self.gyro_data_handler)

    # acc callback function
    def acc_data_handler(self, ctx, data):
        print("ACC: %s -> %s" % (self.device.address, parse_value(data)))
        self.samples+= 1

    # gyro callback function
    def gyro_data_handler(self, ctx, data):
        print("GYRO: %s -> %s" % (self.device.address, parse_value(data)))
        self.samples+= 1

# init
states = []
# connect to all mac addresses in arg
for i in range(len(sys.argv) - 1):
    d = MetaWear(sys.argv[i + 1])
    d.connect()
    print("Connected to " + d.address + " over " + ("USB" if d.usb.is_connected else "BLE"))
    states.append(State(d))

# configure all metawears
for s in states:
    print("Configuring device")
    libmetawear.mbl_mw_settings_set_connection_parameters(s.device.board, 7.5, 7.5, 0, 6000)
    sleep(1.5)

    # config acc
    #libmetawear.mbl_mw_acc_set_odr(s.device.board, 50.0) # Generic call
    libmetawear.mbl_mw_acc_bmi160_set_odr(s.device.board, AccBmi160Odr._50Hz) # BMI 160 specific call
    libmetawear.mbl_mw_acc_bosch_set_range(s.device.board, AccBoschRange._4G)
    libmetawear.mbl_mw_acc_write_acceleration_config(s.device.board)

    # config gyro
    libmetawear.mbl_mw_gyro_bmi160_set_range(s.device.board, GyroBoschRange._1000dps);
    libmetawear.mbl_mw_gyro_bmi160_set_odr(s.device.board, GyroBoschOdr._50Hz);
    libmetawear.mbl_mw_gyro_bmi160_write_config(s.device.board);

    # get acc signal and subscribe
    acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(s.device.board)
    libmetawear.mbl_mw_datasignal_subscribe(acc, None, s.accCallback)

    # get gyro signal and subscribe
    gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(s.device.board)
    libmetawear.mbl_mw_datasignal_subscribe(gyro, None, s.gyroCallback)

    # start acc
    libmetawear.mbl_mw_acc_enable_acceleration_sampling(s.device.board)
    libmetawear.mbl_mw_acc_start(s.device.board)

    # start gyro
    libmetawear.mbl_mw_gyro_bmi160_enable_rotation_sampling(s.device.board)
    libmetawear.mbl_mw_gyro_bmi160_start(s.device.board)

# sleep 10 s
sleep(10.0)

# breakdown metawears
for s in states:
    # stop acc
    libmetawear.mbl_mw_acc_stop(s.device.board)
    libmetawear.mbl_mw_acc_disable_acceleration_sampling(s.device.board)

    # stop gyro
    libmetawear.mbl_mw_gyro_bmi160_stop(s.device.board)
    libmetawear.mbl_mw_gyro_bmi160_disable_rotation_sampling(s.device.board)

    # unsubscribe acc
    acc = libmetawear.mbl_mw_acc_get_acceleration_data_signal(s.device.board)
    libmetawear.mbl_mw_datasignal_unsubscribe(acc)

    # unsubscribe gyro
    gyro = libmetawear.mbl_mw_gyro_bmi160_get_rotation_data_signal(s.device.board)
    libmetawear.mbl_mw_datasignal_unsubscribe(gyro)

    # disconnect
    libmetawear.mbl_mw_debug_disconnect(s.device.board)

# download recap
print("Total Samples Received")
for s in states:
    print("%s -> %d" % (s.device.address, s.samples))

OSError: [WinError -529697949] Windows Error 0xe06d7363